In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

In [3]:
train = pd.read_csv('../data/train_xy.csv')

In [4]:
test = pd.read_csv('../data/test_all.csv')

In [6]:
x_train = train.drop(['cust_group','y','\ufeffcust_id'],axis =1)
x_test = test.drop(['cust_group','cust_id'],axis=1)
x_train.shape
x_test.shape

(10000, 157)

In [7]:
x = pd.concat([x_train,x_test])
x.shape

(25000, 157)

In [8]:
Y_train = train['y']

In [9]:
for i in range(96,158):
    col = 'x'+'_'+str(i)
    dummies_df = pd.get_dummies(x[col]).rename(columns=lambda x: col + str(x))
    x = pd.concat([x, dummies_df], axis=1)

In [10]:
train_X = x[0:15000]
test_X = x[15000:25000]
print(train_X.shape)
print(test_X.shape)

(15000, 355)
(10000, 355)


In [11]:
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [13]:
X_train,X_val,y_train,y_val= train_test_split(train_X,Y_train,test_size=0.2,random_state=2)

In [14]:
gbm = XGBClassifier( n_estimators= 100, max_depth= 4, min_child_weight= 2, gamma=0.9, subsample=0.8, 
                        colsample_bytree=0.8, objective= 'binary:logistic', nthread= -1, scale_pos_weight=1).fit(X_train, y_train)

In [15]:
predictions = gbm.predict_proba(X_val)
pre = predictions[:,1]
val_auc = metrics.roc_auc_score(y_val,pre)#验证集上的auc值
print(val_auc)

0.80642048092


In [16]:
preds = gbm.predict_proba(test_X)
pred = preds[:,1]
pred.shape

(10000,)

In [18]:
Submission = pd.DataFrame({'cust_id': test['cust_id'], 'pred_prob': pred})
Submission.to_csv('../result/xgb.csv',index=False)